In [1]:
!pip install -e .

Obtaining file:///home/ec2-user/SageMaker/faropt/sdk
  Attempting uninstall: faropt
    Found existing installation: faropt 0.1.2
    Uninstalling faropt-0.1.2:
      Successfully uninstalled faropt-0.1.2
  Running setup.py develop for faropt
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
from faropt import FarOpt

In [3]:
fo = FarOpt()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:root:FarOpt backend is ready!


In [4]:
!pygmentize ../tests/src/main.py

"""Capacited Vehicles Routing Problem (CVRP)."""

# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from utils import *
# [END import]


# [START data_model]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
           

In [5]:
fo.configure('../tests/srcga/')

INFO:root:Listing project files ...
INFO:root:Configured job!


../tests/srcga/ ['tsp'] ['main.py', 'gr17.json']
main.py
gr17.json
../tests/srcga/tsp [] ['gr17.json']
gr17.json


In [6]:
fo.submit()

INFO:root:Submitting job
INFO:root:Submitted job! id: 2020-11-11-23-22-34-72f8cb1b-bfcf-4894-843e-57ab51f93957


In [7]:
fo.list_jobs()

jobid:2020-11-11-23-15-54-f3001776-6dbb-4bbf-8a18-a9575cd21cdb | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-11-23-15-54-f3001776-6dbb-4bbf-8a18-a9575cd21cdb/source.zip
jobid:2020-11-11-23-20-28-e20a1366-119c-40b1-ad79-f2fee3f85435 | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-11-23-20-28-e20a1366-119c-40b1-ad79-f2fee3f85435/source.zip
jobid:2020-11-11-23-22-34-72f8cb1b-bfcf-4894-843e-57ab51f93957 | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-11-23-22-34-72f8cb1b-bfcf-4894-843e-57ab51f93957/source.zip
jobid:2020-11-11-23-17-21-37e71081-5c1e-49e5-af89-7607e2293114 | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-11-11-23-17-21-37e71081-5c1e-49e5-af89-7607e2293114/source.zip


In [8]:
import time
while fo.primary_status()!='STOPPED':
    print(fo.primary_status())
    time.sleep(3)

PROVISIONING
PROVISIONING
PROVISIONING
PROVISIONING
PENDING
PENDING
PENDING
PENDING
PENDING
PENDING
RUNNING
RUNNING
RUNNING
RUNNING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...


In [9]:
fo.logs()

INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


1605136993093 |  Starting FarOpt backend
1605136993093 | ███████╗ █████╗ ██████╗  ██████╗ ██████╗ ████████╗
1605136993093 | ██╔════╝██╔══██╗██╔══██╗██╔═══██╗██╔══██╗╚══██╔══╝
1605136993093 | █████╗  ███████║██████╔╝██║   ██║██████╔╝   ██║   
1605136993093 | ██╔══╝  ██╔══██║██╔══██╗██║   ██║██╔═══╝    ██║   
1605136993093 | ██║     ██║  ██║██║  ██║╚██████╔╝██║        ██║   
1605136993093 | ╚═╝     ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝        ╚═╝   
1605136993093 | ---------------------------------------------------------------
1605136993093 | Downloading source
1605136993093 | Looking for source.zip in  faropt-s3bucketfbfa637e-jauzfgds5cug / 2020-11-11-23-22-34-72f8cb1b-bfcf-4894-843e-57ab51f93957
1605136993093 | Downloaded source.zip, uncompressing
1605136993093 | ---------------------------------------------------------------
1605136993093 | ['gr17.json', 'tsp', 'main.py', 'utils.py', 'task.py']
1605136993860 | gen	nevals	avg    	std    	min 	max 
1605136993860 | 0  	300   	4719.15	431.944	325